In [3]:
import pandas as pd

# Importing the current survey as a csv file that the respondent filled out (will need to adjust depending on the file location)
# Right now I donwload it to my PC to upload it.
survey = pd.read_csv('responses.csv', encoding = "ISO-8859-1") 

In [4]:
# Dropping the unneccesary columns from the survey that we don't need
survey = survey.drop(survey.columns[[0, 17, 18, 19, 20]], axis = 1)

In [5]:
# Rephrasing the column names of the survey so they match with the Comparative Brands Google Sheet made by Lex and Maggie
# IF THE SURVEY QUESTIONS CHANGE, THESE MUST BE UPDATED!
dict = {'Hello mountain, I will climb you now.': 'Q1_1',
       'Sweat, sports, all that jazz.': 'Q1_2',
       'Dinner and drinks':'Q1_3',
       'Take me to the beach!':'Q1_4',
       'Want shoes, get shoes': 'Q1_5',
       'Would you like tee-shirts? We have many.': 'Q1_6',
       "Perfect, let's talk budget. What feels right to spend on a quality item?": 'Q2',
       '*Chemical Use*: Reducing amount of harmful chemicals used in production': 'Q3_1',
       '*Water Conservation*: Reducing amount of water used in production': 'Q3_2',
       '*Carbon Emissions/Energy Use*: Reducing greenhouse gas emissions and energy use from production': 'Q3_3',
       '*Peoples well-being*: Fair treatment of employees and community impacted by production': 'Q3_4',
       '*Animal welfare*: Avoiding using animal-derived materials. Dont use the textile that needs to kill animals or source from wild animals.': 'Q3_5',
       '*Product quality, reuse & recycle*: Making products built to last and supporting reuse/recycling of materials and products': 'Q3_6',
       '*Knowing supply chain*: Knowing how a product is made from start to finish': 'Q3_7',
       '*Transparency*: Publicly accessible and truthful information about company policies and practices': 'Q3_8',
       '2 more left... When you look in your closet what style of clothes do you like to see?': 'Q4'}

# Renaming the columns
survey = survey.rename(columns = dict)

In [6]:
import numpy as np

# Because of how the survey is designed (possible to select multiple inputs), we need to turn the blank values into zeros and the selected  values into 1.
survey['Q1_1'] = np.where(survey['Q1_1'].isna() == True, 0, 1)
survey['Q1_2'] = np.where(survey['Q1_2'].isna() == True, 0, 1)
survey['Q1_3'] = np.where(survey['Q1_3'].isna() == True, 0, 1)
survey['Q1_4'] = np.where(survey['Q1_4'].isna() == True, 0, 1)
survey['Q1_5'] = np.where(survey['Q1_5'].isna() == True, 0, 1)
survey['Q1_6'] = np.where(survey['Q1_6'].isna() == True, 0, 1)

survey['Q3_1'] = np.where(survey['Q3_1'].isna() == True, 0, 1)
survey['Q3_2'] = np.where(survey['Q3_2'].isna() == True, 0, 1)
survey['Q3_3'] = np.where(survey['Q3_3'].isna() == True, 0, 1)
survey['Q3_4'] = np.where(survey['Q3_4'].isna() == True, 0, 1)
survey['Q3_5'] = np.where(survey['Q3_5'].isna() == True, 0, 1)
survey['Q3_6'] = np.where(survey['Q3_6'].isna() == True, 0, 1)
survey['Q3_7'] = np.where(survey['Q3_7'].isna() == True, 0, 1)
survey['Q3_8'] = np.where(survey['Q3_8'].isna() == True, 0, 1)

# Creating categories out of the questions where you can only select one input
survey['Q2'] = np.where(survey['Q2'] == 'Less than $50', 0, survey['Q2'])
survey['Q2'] = np.where(survey['Q2'] == '$50-$100', 1, survey['Q2'])
survey['Q2'] = np.where(survey['Q2'] == '$100-$150', 2, survey['Q2'])
survey['Q2'] = np.where(survey['Q2'] == 'Depends on the item', 99, survey['Q2'])
survey['Q2'] = np.where(survey['Q2'] == "Can't put a price on quality", 3, survey['Q2'])

survey['Q4'] = np.where(survey['Q4'] == 'Generally for the gentlemen', 0, survey['Q4'])
survey['Q4'] = np.where(survey['Q4'] == 'More often for the ladies', 1, survey['Q4'])
survey['Q4'] = np.where(survey['Q4'] == 'I love it all', 99, survey['Q4'])

In [7]:
# Importing the Comp Brands for Email MVP google sheet put together by Maggie.
# Found here: https://docs.google.com/spreadsheets/d/1OuYSawjnMFTkVqO9vuBNDnqBaa5baVXFghCQfaXp3Ww/edit?usp=sharing
# Right now I download it to my PC to upload it.
df = pd.read_csv('Comp.csv', encoding = "ISO-8859-1")

In [8]:
# First step is to filter out by price.
# If the survey indicates they want to spend X dollars, we filter for only the brands with X dollars
if survey.loc[0]['Q2'] == 0:
    df2 = df[(df['Q2'] == 0) | (df['Q2'] == 99)]
elif survey.loc[0]['Q2'] == 1:
    df2 = df[(df['Q2'] == 1) | (df['Q2'] == 99)]
elif survey.loc[0]['Q2'] == 2:
    df2 = df[(df['Q2'] == 2) | (df['Q2'] == 99)]
elif survey.loc[0]['Q2'] == 3:
    df2 = df[(df['Q2'] == 3) | (df['Q2'] == 99)]
else:
    df2 = df

# Now we filter based on gender.
# If the survey indcates they want to buy clothes for X gender, we filter for only the brands for X gender.
if survey.loc[0]['Q4'] == 0:
    df2 = df2[(df2['Q4'] == 0) | (df2['Q4'] == 99)] 
elif survey.loc[0]['Q4'] == 1:
    df2 = df2[(df2['Q4'] == 1) | (df2['Q4'] == 99)]
else:
    df2 = df2

# Note that the new comparative brand filtred dataframe is now called df2

In [9]:
# Manipulating the comparative brands sheet to use in the KNN algorithm
col = [col for col in df2 if col.startswith('Q')]
training = df2[col] # We want to extrat only these columns

# These are the columns we want to include in the final output
# Note these can be adjusted depending on the info we want to retain.
names = df2[['Brand', 'Info', 'Item', 'Price','Link','Contact']]

# Dropping the Questions we already filtered on.
training = training.drop(['Q2','Q4'], axis = 1)
survey = survey.drop(['Q2','Q4'], axis = 1)

In [10]:
# Resetting the indicies to go from 1...n
names.reset_index(drop=True, inplace=True)
training.reset_index(drop=True, inplace=True)

In [11]:
# Using nearest neighbors algorithm
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors = 3) # setting to 3 will output the three closest comparisons
neigh.fit(training) # Fitting on the comparative brands data
print(neigh.kneighbors(survey)) # Printing the three closest from the respondent survey

(array([[2.23606798, 2.23606798, 2.44948974]]), array([[ 2, 22,  1]], dtype=int64))


In [12]:
three_closest = neigh.kneighbors(survey) # Using the algorithm to extract three closest
three_closest = three_closest[1] # We want only the first index (Euclidean distance)

# For each of the three closest brand comparisons we want to print out the information contained in names (these can be adjusted in cell 7).
for i in three_closest:
    output = names.iloc[i]

In [13]:
# View the final dataframe that is produced (called output)
# This information can be changed by changing the names in cell 7.
output

,Brand,Info,Item,Price,Link,Contact
2,Ministry of Supply,On demand production using zero waste techno...,Juno Blouse,$125.00,https://www.ministryofsupply.com/products/wome...,q@ministryofsupply.com
22,Ministry of Supply,On demand production using zero waste techno...,Apolo Brushed Shirt,$125.00,https://www.ministryofsupply.com/products/mens...,q@ministryofsupply.com
1,Mate the Label,Eliminated all plastic from consumer packagi...,Organic Cotton Long Sleeve Crop,$68.00,https://matethelabel.com/collections/organic-j...,affiliates@matethelabel.com
